# **Contexto**

Nesse desafio, quero criar um sistema de busca em bulas de remédios dentro de um acervo. Para que eu consiga localizar quais remédios resolvem os sintomas do paciente, permitindo que o Gemini AI contribua também com informações de contra-indicações ou sugestão de outros remédios que não estejam no acervo.

---

*Instalação do SDK do Google*


In [36]:
# Install
!pip install -q -U google-generativeai

*Importação do Numpy (ferramenta matemática), Pandas (sistema tabular), PyPDF (biblioteca capaz de extrair informações de documentos em PDF) e Google Gemini*

In [38]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

google_api_key = userdata.get('COLE A API KEY')
genai.configure(api_key=google_api_key)

*Trazer a lista de modelos de embedding*

In [ ]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

*Conteúdo dos documentos que serão buscados*

*   *Bula do Nimesulida*
*   *Bula do Novalgina*
*   *Bula do Tylenol*

*Gostaria de ter colocado mais bulas, mas tive problema com limitação de payload para leitura dos documentos e não soube resolver.*

In [66]:
#Listagem de documentos que serão buscados

def ler_bula_nimesulida():
    with open("bula_nimesulida.txt", "r", encoding="utf-8") as arquivo_nimesulida:
        texto_nimesulida = arquivo_nimesulida.read()
        return texto_nimesulida

def ler_bula_novalgina():
    with open("bula_novalgina.txt", "r", encoding="utf-8") as arquivo_novalgina:
        texto_novalgina = arquivo_novalgina.read()
        return texto_novalgina

def ler_bula_tylenol():
    with open("bula_tylenol.txt", "r", encoding="utf-8") as arquivo_tylenol:
        texto_tylenol = arquivo_tylenol.read()
        return texto_tylenol

DOCUMENT1 = {
    "Título": "Nimesulida",
    "Conteúdo": ler_bula_nimesulida()
}
DOCUMENT2 = {
    "Título": "Novalgina",
    "Conteúdo": ler_bula_novalgina()
}
DOCUMENT3 = {
    "Título": "Tylenol",
    "Conteúdo": ler_bula_tylenol()
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

*Criar um Dataframe e definição dos nomes das colunas*

In [ ]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

*Definição do modelo de embedding*

In [42]:
model = 'models/embedding-001'

*Criando embedding para os documentos, usando a API do Google*




In [43]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="retrieval_document")["embedding"]

*Gerando embeddings para cada linha do seu DataFrame*

In [ ]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"][:3000]), axis=1)
df

*Realizando uma busca semântica nos documentos, usando embeddings de texto.*

In [45]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="retrieval_query")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

*Define a consulta desejada e imprime o resultado com um limite de 200 caracteres.*

In [ ]:
consulta = "CONSULTA A SER FEITA
trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho[:200])

*Configurações da IA generativa*

In [ ]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0.5,
}

*Adaptação do texto, para que a leitura fique mais "palatável".*

In [ ]:
prompt = f"Reescreva esse texto de uma forma mais fácil de ler, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)